In [35]:
import openai
import pandas as pd

openai.api_key = "..." # put the api key here, it can't be pushed to github

donors = pd.read_excel("Annual Donations Made.xlsx", sheet_name="donors")

In [45]:
def classify_names_in_batch(names):
    prompt = "Classify the following names as either of the following precise codes 'company' or 'individual':\n\n"
    prompt += "\n".join(names)
    
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=len(names) * 5  # Adjust based on expected response length
    )
    classifications = response.choices[0].text.strip().split('\n')
    return classifications

def classify_names_in_batches(dataframe, column_name, batch_size=20, num_batches=None):
    names = dataframe[column_name].tolist()
    batches = [names[i:i + batch_size] for i in range(0, len(names), batch_size)]
    
    if num_batches:
        batches = batches[:num_batches]
    
    for batch in batches:
        classifications = classify_names_in_batch(batch)
        # Update your DataFrame with the classifications
        for name, classification in zip(batch, classifications):
            dataframe.loc[dataframe[column_name] == name, 'Type'] = classification.strip().lower()

# Classify names in batches, limit to 2 batches for testing
#classify_names_in_batches(donors, 'Donor Name', batch_size=3, num_batches=10)

count = 0
def classify_name(name):
    global count
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",
        prompt=f"Based on your knowledge of Australian company names, classify the following name as either 'company' or 'individual': {name}",
        max_tokens=5
    )
    count += 1
    classification = response.choices[0].text.strip().lower()
    print(count, name, classification)
    return classification

donors['Type'] = donors['Donor Name'].apply(classify_name)

# Print the updated DataFrame
print(donors.head())

1 100 Percent Events Pty Ltd company
2 117 York St Pty Ltd company
3 26 Summers Pty Ltd company
4 3 Oceans Property Pty Ltd company
5 321 Warrigal Road Pty Ltd (formerly Shannons Corporation Pty Ltd )

company
6 41 Hindmarsh Square Developments Pty Ltd 'company'
7 55 Norton Street Pty Ltd company
8 Australia Pacific Airports Corporation company
9 Australian Automobile Association company
10 Australian Automotive Dealer Association company
11 679 Bourke St Pty Ltd company
12 88 Terrace Road Pty Ltd  company
13 Australian Finance Industry Association company
14 Australian Forest Products Association company
15 Australian Hotels Association - Federal Office company
16 Australian Hotels Association (N.S.W.) company
17 Australian Hotels Association (SA Branch) company
18 A & E Edgecombe Nominees Pty Ltd company
19 A & K (NT) Pty Ltd company
20 A J Bush & Sons (Manufactures) Pty Ltd company
21 A Little Something Else Pty Ltd company
22 Barlow, Lisa Jane individual
23 A Raptis & Sons Pty Ltd 

In [48]:
count = 0
def get_industry_sector(company_name):
    global count
    prompt = f"Identify the industry sector for the following Australian company: {company_name}. If the industry sector is not known, respond with 'Unknown'."
    
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=10  # Adjust based on expected response length
    )
    industry_sector = response.choices[0].text.strip()
    count += 1
    print(count, company_name, industry_sector)
    return industry_sector

# Apply the function to populate the 'Industry Sector' column for company-type entries
donors['Industry Sector'] = donors.apply(lambda row: get_industry_sector(row['Donor Name']) if row['Type'] == 'company' else None, axis=1)

donors.to_csv('donors_output.csv', index=False)

1 100 Percent Events Pty Ltd Event management and planning.
2 117 York St Pty Ltd Unknown
3 26 Summers Pty Ltd Unknown
4 3 Oceans Property Pty Ltd Unknown
5 55 Norton Street Pty Ltd Unknown
6 Australia Pacific Airports Corporation Airport Operations/Aviation
7 Australian Automobile Association Automotive/Transportation
8 Australian Automotive Dealer Association Automotive/ Automotive Retail Industry
9 679 Bourke St Pty Ltd Unknown
10 88 Terrace Road Pty Ltd  Unknown
11 Australian Finance Industry Association Financial Services/Finance
12 Australian Forest Products Association The industry sector for Australian Forest Products Association is
13 Australian Hotels Association - Federal Office Hospitality and Tourism Industry
14 Australian Hotels Association (N.S.W.) Hospitality/Hotel industry
15 Australian Hotels Association (SA Branch) Hospitality/Tourism.
16 A & E Edgecombe Nominees Pty Ltd Unknown
17 A & K (NT) Pty Ltd Unknown
18 A J Bush & Sons (Manufactures) Pty Ltd Meat and Poultry 

In [47]:
print(donors['Type'].value_counts())

Type
company                3785
individual             2032
'company'                49
.\n\ncompany             31
'individual'             17
                       ... 
line\n\ncompany           1
the given name "          1
this name is a            1
republic\n\ncompany       1
ia\n\ncompany             1
Name: count, Length: 178, dtype: int64


In [46]:
donors.to_csv('donors_output.csv', index=False)

In [66]:
df = pd.read_csv('donors_output.csv',on_bad_lines='warn')


SyntaxError: invalid syntax. Perhaps you forgot a comma? (7005213.py, line 1)